In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision.transforms import v2

In [ ]:
transform = v2.Compose(
    [v2.Resize((256, 256),antialias=True), # Scaling the images to 256x256 pixels
     v2.ToImageTensor(), # Converting the images to 5D tensors
     v2.ConvertImageDtype(), 
     v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Normalizing the tensors to have a mean of 0.5 and standard deviation of 0.5

dataset = torchvision.datasets.ImageFolder(root='data', transform=transform)

np.random.seed(10)  # Assign a seed to get the complementary testing set of the set used in training the model
n_samples = len(dataset)
n_train = int(0.8 * n_samples)
n_test = n_samples - n_train

classes = ('Common Wheat','Aegilops cylindrica L.', 'Avena fatua L.', 'Bromus sterilis L.', 
            'Bromus tectorum L.', 'Echinochloa crus-galli (L.) Beauv')


In [ ]:
# Separate the data into test and training sets and return them
def randomSetSeparation():
    batch_size = 20
    
    trainset, testset = torch.utils.data.random_split(dataset, [n_train, n_test])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

    return trainloader, testloader

In [ ]:
# Setting the class for specific CNN architecture
# Two convolutional layers each followed by a max pooling layer and three fully connected layers
class Net(nn.Module):
    
    # The constructor for the network 
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 62 * 62, 80)
        self.fc2 = nn.Linear(80, 42)
        self.fc3 = nn.Linear(42, 6)

    # Forward propagation
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        return x

net = Net() # Intitialization of the network

In [ ]:
PATH = 'weed_classifier.pth' # Path to do the model
print(f'Model: {PATH}')
net.load_state_dict(torch.load(PATH)) # Load the model

print(f'Model success rates in percentages to the nearest integer for each trial:')

for i in range(30):

    trainloader, testloader = randomSetSeparation()

    correct = 0.0
    total = 0.0

    # Forward propagat,ion
    with torch.no_grad():
        for data in testloader:
            # Getting the inputs
            images, labels = data
            
            # Calculate the outputs by feeding images to the network
            outputs = net(images) 
            
            # Choosing the the class with the highest value as prediction
            _, predicted = torch.max(outputs.data, 1)

            # Calculating the number of test images and correct predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'{100.0 * correct // total}') # Printing the performance


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')